In [33]:
import pymc as pm
import numpy as np
import matplotlib.pyplot as plt
#import utils
%matplotlib inline

# the same as soft evidence fact
f_1_of_10000 = pm.Categorical("f_1_of_10000", [1, 0])
print(f_1_of_10000.random())
I_am_HIV_positive = pm.Categorical("I_am_HIV_positive", [0.0001, 0.9999])
I_dont_know = pm.Categorical("I_dont_know", [0.5, 0.5])

@pm.deterministic
def p_I_am_HIV_positive_soft(f_1_of_10000=f_1_of_10000, I_am_HIV_positive=I_am_HIV_positive, I_dont_know=I_dont_know):
    if (f_1_of_10000 == 1):
        return I_am_HIV_positive
    else:
        return I_dont_know

print(p_I_am_HIV_positive_soft.value)

@pm.stochastic(observed=True)
def HIV_test_false_positive(value=1, p_I_am_HIV_positive_soft=p_I_am_HIV_positive_soft):
    if ( p_I_am_HIV_positive_soft != 0 and p_I_am_HIV_positive_soft != 1 ):
        return -np.inf
    elif ( p_I_am_HIV_positive_soft == 0 ):
        prob = 0.999 if value == 1 else 0.001 # 0.999 - prob that test is true if a am really HIV
    else:
        prob = 0.001 if value == 1 else 0.999 # 0.001 - prob that test is true if a am not really HIV
    return np.log(prob)
     
model = pm.Model([f_1_of_10000, I_am_HIV_positive, I_dont_know, p_I_am_HIV_positive_soft, HIV_test_false_positive])
#utils.display_graph(model)
mcmc = pm.MCMC(model)
mcmc.sample(20000, 2000)

alpha_samples = mcmc.trace("p_I_am_HIV_positive_soft")[:]

print(sum(alpha_samples)/20000)
# f_HIV_test_false_positive = pm.math.switch(
#     p_I_am_HIV_positive_soft == 0,
#     pm.Categorical("test_if_I_am_not_HIV", [0.999, 0.001]),
#     pm.Categorical("test_if_I_am_HIV", [0.001, 0.999]),
#     observed=1
# )

0
0
 [-----------------100%-----------------] 20000 of 20000 complete in 18.9 sec0.00145


In [ ]:
def age_logp(age):
    if age < 0 or age > 110:
        return -np.inf
    elif 0 <= age < 20:
        prob = age_dist['0-20']
    elif 20 <= age < 40:
        prob = age_dist['20-40']
    elif 40 <= age < 60:
        prob = age_dist['40-60']
    elif 60 <= age < 80:
        prob = age_dist['60-80']
    elif age >= 80:
        prob = age_dist['80-inf']
    return np.log(prob)


@pymc.stochastic
def age(value=0):
    def logp(value):
        return age_logp(value)